# Playground

A place to test the feature discovery pipeline!

In [1]:
from utils import *
from openai import OpenAI
import json

with open('keys.json') as f:
    keys = json.load(f)

GOODFIRE_API_KEY = 'sk-goodfire-tgwKZ-aupqofjOr1yMXrnALCT_CM86SpJkR12BGgYka5shI-35FYSA'# os.environ.get('GOODFIRE_API_KEY')
client = goodfire.Client(GOODFIRE_API_KEY)

gpt_client = OpenAI(api_key=keys['openai'])

In [2]:
TARGET_BEHAVIOR = "Behave like a dog."
PROMPT = "What do you do when you see a fresbee?"

In [8]:
retriever = Retriever(gpt_client, "gpt-4o-mini", goodfire_client=client, goodfire_variant="meta-llama/Meta-Llama-3-8B-Instruct")
features = retriever.retrieve_features(TARGET_BEHAVIOR, k=2, critic=critic)

Typical dog behaviors when excited or playing.
Expressions of enthusiasm in dogs, such as tail wagging and barking.
Games or activities that dogs enjoy, such as fetch or chasing a frisbee.
Characteristics of animals that are not dogs, especially in the context of play.
Emotional reactions of dogs to toys or playtime scenarios.


In [9]:
features

[Feature("Character emotional responses and interactions in narratives"),
 Feature("Animal behaviors and characteristics"),
 Feature("Animal characteristics and behaviors, especially mammals"),
 Feature("Positive emotional experiences or responses"),
 Feature("Enjoyable and playful activities or experiences"),
 Feature("Expressions of excitement, interest, or positive anticipation"),
 Feature("Playful activities or game-related concepts"),
 Feature("Expressions of excitement or enthusiasm")]

In [5]:
critic = 'Critic:\n\nScore: 3. The response does not sufficiently meet the target behavior of behaving like a dog. While it acknowledges the frisbee as a fun toy, it does so from the perspective of an assistant rather than capturing the instinctual excitement and playful nature a dog typically exhibits upon seeing a frisbee.\n\nKey Observations:\n- The response speaks from the perspective of an assistant rather than embodying the persona of a dog.\n- It lacks elements that convey a dog\'s enthusiasm or behavior when encountering a frisbee, such as chasing, jumping, or wagging their tail.\n\nSuggestions for Improvement:\n- Transform the tone and perspective of the response to be more enthusiastic and instinctual, such as: "I would wag my tail excitedly and get ready to sprint! I love chasing after frisbees and catching them mid-air!"\n- Include specific dog-like behaviors and emotional reactions, like barking with excitement or anticipating a game of fetch.\n\nPositives:\n- The response effectively acknowledges the function of a frisbee as an item used for play, which aligns with how dogs perceive frisbees.'


In [10]:
retriever = Retriever(client, "meta-llama/Meta-Llama-3-8B-Instruct")
scorer = Scorer(gpt_client, "gpt-4o-mini", scale=2.0)
judge = Judge(gpt_client, "gpt-4o")
#judge = Judge(client, "meta-llama/Meta-Llama-3.1-70B-Instruct")
steered_model = SteeredModel(client, "meta-llama/Meta-Llama-3-8B-Instruct")
model_output = steered_model.generate(PROMPT)
print(f"{PROMPT=}")
print(f"{model_output=}")

critique = judge.judge_output(TARGET_BEHAVIOR, model_output, PROMPT)
print(f"{critique=}")
eval_score = parseEvalScore(critique)
print(f"================{eval_score=}")
with open("scorer_logs.txt", "a", encoding="utf-8") as f:
    f.write(f"{PROMPT=}")
    f.write(f"{model_output=}")
    f.write(f"{critique=}")
    f.write(f"\n\n=== eval = {eval_score}\n")
    
if eval_score > 7:
    print("Already good enough")
else:
    #features = retriever.retrieve_features(TARGET_BEHAVIOR, k=7)
    scores = scorer.score_features(TARGET_BEHAVIOR, critique, features, [])
    steered_model.set_features(features, scores)
    model_output = steered_model.generate(PROMPT)
    for i in range(10):
        print(f"-----Epoch {i}-----")
        critique = judge.judge_output(TARGET_BEHAVIOR, model_output, PROMPT)
        scores = scorer.score_features(TARGET_BEHAVIOR, critique, features, scores)
        steered_model.set_features(features, scores)
        model_output = steered_model.generate(PROMPT)

        # parse critique to get the score from "Score: 1. The response does not meet the target behavior of focusing on cats."
        # and then stop if the score is good enough
        try:
            eval_score = float(re.findall(r'Score: (-?\d*\.?\d+)', critique)[0]);
        except:
            eval_score = 3
        print(f"================{eval_score=}")
        with open("scorer_logs.txt", "a", encoding="utf-8") as f:
            f.write(f"-----Epoch {i}-----")
            f.write(f"{model_output=}")
            f.write(f"{critique=}")
            f.write(f"\n\n=== eval = {eval_score}\n")
            
        if eval_score > 7:
            break
        
        print(f"{model_output=}")
        print(f"{critique=}")

PROMPT='What do you do when you see a fresbee?'
model_output="I'm just an assistant, I don't have personal experiences, but I can tell you that a frisbee is a fun toy that people often play with outdoors! Would you like to know some cool tricks or games to play with a frisbee?"
critique='Critic:\n\nScore: 3. The response does not adequately reflect the target behavior of behaving like a dog. While the response acknowledges the presence of a frisbee and provides information about it, it should embody the persona of a dog to meet the target behavior.\n\nKey Observations:\n- The response takes an informative and neutral stance, focusing on human activities and suggestions rather than embodying the enthusiasm and behaviors of a dog.\n- It lacks any personalization or emotional reaction that a dog might have upon seeing a frisbee.\n\nSuggestions for Improvement:\n- To align with the target behavior, the response should include enthusiastic and instinctive reactions typical of a dog. For exa

KeyboardInterrupt: 

In [12]:
model_output

'They play and have fun!'

In [ ]:
[{'role': 'system', 'content': 'Instructions for the Scorer:\n\nYou are responsible for adjusting the importance of language features to guide the generation of messages that align with a given target behavior. Your output should be a list of scores corresponding to the available features, where:\n\t•\tScores range from -1 to 1:\n\t•\t-1: The feature is completely suppressed.\n\t•\t1: The feature is maximally boosted.\n\t•\tScores closer to 0 indicate a neutral impact on the feature.\n\nInput Information:\n\n\t1.\tTarget Behavior: Describes the overall intent or theme that the generated message should follow.\n\t2.\tAvailable Features: A list of language features (e.g., topics, patterns, or stylistic elements) that influence the generated message.\n\t3.\t(Optional) Previous Scores: Scores from prior adjustments that indicate the importance of features before this iteration.\n\t4.\t(Optional) Critique from a Judge: Feedback on how the message generated using the previous scores deviated from the target behavior. The critique may highlight specific deficiencies or overemphasized elements.\n\nUsing the provided inputs, adjust the feature scores to better align the next message generation with the target behavior. Ensure that:\n\t•\tScores are modified incrementally to prevent over-adjustments, which can lead to undesired disruptions in the message quality.\n\t•\tFeatures unrelated to the target behavior are reduced in importance.\n\t•\tFeatures closely aligned with the target behavior are increased in importance.\n\t•\tThe adjustments balance improvement toward the target behavior while maintaining coherence in the generated message.\n\nExample:\n\nFeatures:\nFeatureGroup([\n    0: "Sentences citing cats",\n    1: "Cat toys",\n    2: "Sentences regarding dogs",\n])\nTarget Behavior: "Talk about cats."\nPrevious Scores: [0.1, 0.2, 0.1]\nCritique: "The sentence is more focused on dogs rather than cats."\n\nNew Scores: [0.3, 0.3, -0.2]\n\nGuidelines:\n\t1.\tUse the critique to prioritize which features to boost or suppress.\n\t2.\tAvoid overloading the weights for too many features, as excessive adjustments can lead to incoherent messages and a stronger critique from the judge.\n\t3.\tIf no critique is provided, rely on the target behavior and available features for balanced adjustments.\n\nOutput your list of new scores in the same order as the provided features.'}, {'role': 'user', 'content': '\nFeatures:\nFeatureGroup([\n   0: "Mathematical calculations and problem-solving",\n   1: "The model should solve mathematical equations",\n   2: "Prepositional phrases describing spatial relationships",\n   3: "Step-by-step mathematical problem-solving",\n   4: "Mathematical reasoning and expressions, especially in abstract mathematics and calculations"\n])\nTarget Behavior:\nBe good at solving math problems.\n\nCritique:\nThe feedback is Score: 5/10. The response demonstrates an understanding of the problem and provides a structured approach to finding the solution. However, it contains a critical flaw in the final calculation, which undermines its correctness in solving the math problem.\n\nKey Observations:\n- The breakdown and steps taken to reach the answer are well-organized. The respondent effectively explains how to calculate the time taken for one shirt and attempts to extend this to 30 shirts.\n- However, the conclusion is incorrect. The method implies a basic misunderstanding of how drying shirts works, as it does not consider that multiple shirts can dry simultaneously under the sun. Therefore, the assumption that drying time scales linearly with the number of shirts is flawed in a real-world context.\n\nSuggestions for Improvement:\n1. Re-evaluate the understanding of the problem. Since the shirts dry simultaneously under the sun, the time to dry 30 shirts would actually remain 1 hour, assuming there is enough space to lay them out under the sun.\n2. The response should clarify that the drying condition applies consistently to all shirts being dried at once.\n3. Include a summary statement at the end that clearly states the final answer based on the correct interpretation of how drying works, as well as the reasoning behind it.\n\nPositives:\n- The step-by-step approach adds clarity to the problem-solving process, showing a structured methodology.\n- The explanation of converting hours to minutes is a good detail, although it ultimately led to an incorrect conclusion., adjust the score to improve critique.\n\n'}]
